In [1]:
import numpy as np
import pandas as pd

import math
import random
import operator
import itertools

from deap import algorithms,base,creator,tools,gp
from features1 import fill_age_1,add_title

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_x = train.drop("Survived",axis=1)
train_y = train['Survived']

def MungeData(data):
    # Sex
#    data=add_title(data)   
#    data=fill_age_1(data)
#    data.drop('Title', axis=1, inplace=True)
    data.drop(['Ticket', 'Name', 'PassengerId'], inplace=True, axis=1)
    data.Sex.fillna('0', inplace=True)
    data.loc[data.Sex != 'male', 'Sex'] = 0
    data.loc[data.Sex == 'male', 'Sex'] = 1
    # Cabin
    data.Cabin.fillna('0', inplace=True)
    data.loc[data.Cabin.str[0] == 'A', 'Cabin'] = 1
    data.loc[data.Cabin.str[0] == 'B', 'Cabin'] = 2
    data.loc[data.Cabin.str[0] == 'C', 'Cabin'] = 3
    data.loc[data.Cabin.str[0] == 'D', 'Cabin'] = 4
    data.loc[data.Cabin.str[0] == 'E', 'Cabin'] = 5
    data.loc[data.Cabin.str[0] == 'F', 'Cabin'] = 6
    data.loc[data.Cabin.str[0] == 'G', 'Cabin'] = 7
    data.loc[data.Cabin.str[0] == 'T', 'Cabin'] = 8
    # Embarked
    data.loc[data.Embarked == 'C', 'Embarked'] = 1
    data.loc[data.Embarked == 'Q', 'Embarked'] = 2
    data.loc[data.Embarked == 'S', 'Embarked'] = 3
    data.Embarked.fillna(0, inplace=True)
    data.fillna(-1, inplace=True)

    return data.astype(float)
train_x = MungeData(train_x).values.tolist()
test = MungeData(test).values.tolist()
train_y = train_y.tolist()

In [2]:
def proba(data):
    return (1.-(1./(1.+np.exp(-data))))

def predict(prob):
    return np.round(prob)

def protectedDiv(a,b):
    try:
        return a/b
    except ZeroDivisionError:
        return 1
def protectedLog(a):
    x=np.log(a)
    if np.isnan(x):
        return 728
    return x

pset = gp.PrimitiveSetTyped('MAIN',itertools.repeat(float, 8), float)
pset.addPrimitive(protectedLog, [float], float)
pset.addPrimitive(operator.add, [float,float],float)
pset.addPrimitive(operator.sub, [float,float],float)
pset.addPrimitive(operator.mul, [float,float],float)
pset.addPrimitive(protectedDiv, [float,float],float)
pset.addPrimitive(operator.neg, [float],float)
pset.addPrimitive(np.cos,[float],float)
pset.addPrimitive(np.sin,[float],float)
pset.addPrimitive(np.tanh,[float],float)
pset.addPrimitive(np.minimum, [float,float],float)
pset.addPrimitive(np.maximum, [float,float],float)
# Terminals
pset.addEphemeralConstant("rand1", lambda: random.random()*2-1, float)
pset.addEphemeralConstant("rand1000", lambda: round(random.random()*1000),float)
pset.addTerminal(np.pi,float)
pset.addTerminal(2.0,float)
pset.renameArguments(ARG0='PClass')
pset.renameArguments(ARG1='Sex')
pset.renameArguments(ARG2='Age')
pset.renameArguments(ARG3='SibSp')
pset.renameArguments(ARG4='Parch')
pset.renameArguments(ARG5='Fare')
pset.renameArguments(ARG6='Cabin')
pset.renameArguments(ARG7='Embarked')

In [8]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset = pset, min_=1, max_=3)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)
def log_loss(y_true, y_prob):
    n = len(y_true)
    result=0.0
    for i in range(n):
        y_prob[i]=np.minimum(np.maximum(1e-15, y_prob[i]),1-1e-15)
        if y_true[i]:
            result+=np.log(y_prob[i])
        else:
            result+=np.log(1-y_prob[i])
    return 999.0 if np.isnan(result) else -1.0*result/n

def evalFitness(individual):
    func = toolbox.compile(expr=individual)
    return log_loss(train_y,[proba(func(*x)) for x in train_x]),
    
toolbox.register('evaluate', evalFitness)
def staticLimitCrossover(ind1, ind2, heightLimit):
    keepInd1, keepInd2 = toolbox.clone(ind1), toolbox.clone(ind2)
    gp.cxOnePoint(ind1, ind2)
    if ind1.height > heightLimit:
        ind1[:] = keepInd1
    if ind2.height > heightLimit:
        ind2[:] = keepInd2
        
def staticLimitCrossover1(ind1, ind2, heightLimit):
    keepInd1, keepInd2 = toolbox.clone(ind1), toolbox.clone(ind2)
    gp.cxOnePointLeafBiased(ind1, ind2, 0.2)
    if ind1.height > heightLimit:
        ind1[:] = keepInd1
    if ind2.height > heightLimit:
        ind2[:] = keepInd2

def staticLimitMutation(individual, expr, pset, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutUniform(individual, expr,pset)
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation1(individual, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutEphemeral(individual, mode='one')
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation2(individual, pset, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutNodeReplacement(individual, pset)
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation3(individual, pset, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutInsert(individual, pset)
    if individual.height > heightLimit:
        individual[:] = keepInd 

def staticLimitMutation4(individual, heightLimit):
    keepInd = toolbox.clone(individual)
    gp.mutShrink(individual)
    if individual.height > heightLimit:
        individual[:] = keepInd 
        
def selDoubleTournament(individuals, k, fitTournSize, sizeTournSize):
    def _sizeTournament(individuals, tournamentSize):
        chosen = []
        aspirant1 = random.choice(individuals)
        aspirant2 = random.choice(individuals)
        s1, s2 = aspirant1.height, aspirant2.height
        if s1 < s2:
            return aspirant1 if random.random() < tournamentSize / 2.0 else aspirant2
        elif s1 > s2:
            return aspirant2 if random.random() < tournamentSize / 2.0 else aspirant1
        else:
            return random.choice([aspirant1, aspirant2])
    chosen = []
    for i in xrange(k):
        chosen.append(_sizeTournament(individuals, sizeTournSize))
        for j in xrange(fitTournSize - 1):
            aspirant = _sizeTournament(individuals, sizeTournSize)
            if aspirant.fitness > chosen[i].fitness:
                chosen[i] = aspirant
    return chosen 

toolbox.register('selectW', tools.selWorst)
toolbox.register('selectR', tools.selRoulette)
toolbox.register('selectT', tools.selTournament, tournsize=3)
toolbox.register('selectB', tools.selBest)
toolbox.register('selectSPEA', tools.selSPEA2)
toolbox.register('select', selDoubleTournament)


toolbox.register("mate", staticLimitCrossover, heightLimit=23)
toolbox.register("mateL", staticLimitCrossover1, heightLimit=23)
toolbox.register("mutateS",  staticLimitMutation4, heightLimit=23) 
toolbox.register("mutateI",  staticLimitMutation3, pset=pset, heightLimit=23) 
toolbox.register("mutateR",  staticLimitMutation2, pset=pset, heightLimit=23) 
toolbox.register("mutateE",  staticLimitMutation1, heightLimit=23) 
toolbox.register("mutateU", staticLimitMutation, expr=toolbox.expr, pset=pset, heightLimit=23) 



In [9]:
# Elite Approach + New Blood Mechanism (2016)
# Target fitness(want to minimize)
size=200
target = 0.2
gen=0
PR_X = 0.8
PR_M = 0.075


pop = toolbox.population(n=size)
fitnesses = map(toolbox.evaluate, pop)
for ind,fit in zip(pop, fitnesses):
    ind.fitness.values = fit
hof = tools.HallOfFame(1)
kick_start = toolbox.selectB(pop,1)[0]
hof.insert(kick_start)
previous_fitness_df=[]
cleanse = False
while hof[0].fitness.values[0]>target:
    gen+=1   
    # copy best individual
    best=pop[0]
    for ind in pop:
        if ind.fitness>best.fitness:
            best=ind
    
    # natural selection
    selected = toolbox.select(pop,size, 3, 1.4)
    parents = map(toolbox.clone, selected)
    
    # update prior fitness scores and check for criteria for new blood approach 
    previous_fitness_df.append(np.average([ind.fitness.values[0] for ind in selected]))
    if len(previous_fitness_df)>7:
        del previous_fitness_df[0]
    if len(previous_fitness_df)==7 and np.fabs(previous_fitness_df[0]-previous_fitness_df[6])<1e-4:
        print "Reset!!"
        pop1 = toolbox.select(parents, int(round(0.05*size)),3, 1.4)
        pop2 = toolbox.population(int(round(0.95*size)))
        pop11 = map(toolbox.clone,pop1)
        pop22 = map(toolbox.clone,pop2) 
        fitnesses = map(toolbox.evaluate, pop22)
        for ind, fit in zip(pop22, fitnesses):
            ind.fitness.values = fit
        cleanse = True
        parents[:] = map(toolbox.clone,pop11+pop22)
        previous_fitness_df = []
        PR_M+=0.01
        
    print gen, best.fitness.values[0], np.average([ind.fitness.values[0] for ind in parents])
    
    # crossing
    n=PR_X*len(parents)
    n-=n%2
    if cleanse:
        # new blood appraoch
        m = round((random.random()/2)+0.5)
        d1 = map(toolbox.clone, toolbox.selectSPEA(pop22,int(round(m))))
        d2 = map(toolbox.clone, toolbox.selectSPEA(pop11,int(n)-int(round(m))))
        index=0
        for ind1,ind2 in zip(d2[:],d1[:]):
            toolbox.mateL(ind1,ind2)
            index+=1
            del ind1.fitness.values
            del ind2.fitness.values
        for ind1,ind2 in zip(d1[index::2],d1[index+1::2]):
            toolbox.mateL(ind1,ind2)
            del ind1.fitness.values
            del ind2.fitness.values
        crossing = map(toolbox.clone,d1+d2)
        cleanse=False
    else:
        # normal cross breed
        crossing = toolbox.selectSPEA(parents,int(n))
        for ind1,ind2 in zip(crossing[::2],crossing[1::2]):
            toolbox.mateL(ind1,ind2)
            del ind1.fitness.values
            del ind2.fitness.values
    
    p1 = toolbox.select(parents, size-int(n), 3, 1.4)
    p11 = map(toolbox.clone, p1)
    offspring=map(toolbox.clone,p11+crossing)

    # random mutation 
    for ind in offspring:
        if random.random() < PR_M:
            toolbox.mutateU(ind)
            del ind.fitness.values
            
    # re-evaluate fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop[:]=offspring+[best]
    hof.update(pop)
    
        

1 0.571139929369 0.892358491797
2 0.571139929369 0.656975427861
3 0.571139929369 0.608674828977
4 0.569472736693 0.579927906102
5 0.558292383207 0.571197315354
6 0.558119456091 0.570099247871
7 0.558119456091 0.569253092317
8 0.558119456091 0.568400638981
9 0.558119456091 0.567184618314
10 0.558119456091 0.565959826723
11 0.558119456091 0.563873522101
12 0.558119456091 0.561674030586
13 0.550154485878 0.559343381779
14 0.550154485878 0.558329336208
15 0.550154485878 0.558211862667
16 0.550154485878 0.558070846401
17 0.550154485878 0.558075141747
18 0.550154485878 0.557865505411
19 0.550154485878 0.557970567674
20 0.550154485878 0.558002494739
21 0.550154485878 0.558093715203
Reset!!
22 0.550154485878 18.2474604187
23 0.550154485878 0.602745628513
24 0.550154485878 0.567412936709
25 0.550154485878 0.558507327272
26 0.550154485878 0.558102155121
27 0.546667481095 0.558181572259
28 0.546667481095 0.558136824237
29 0.542953313072 0.557866398846
30 0.542953313072 0.55777951598
31 0.54295331

246 0.43669119766 0.439660227248
247 0.43669119766 0.441063487741
248 0.43669119766 0.438593154191
249 0.43669119766 0.439028588879
250 0.43669119766 0.43846027738
251 0.43669119766 0.438271695044
252 0.43669119766 0.440209318342
253 0.43669119766 0.438860367951
254 0.43669119766 0.438713207598
255 0.43669119766 0.442115242858
256 0.43669119766 0.441393817071
257 0.43669119766 0.441173481322
258 0.43669119766 0.438624496114
259 0.436375575593 0.438437526836
260 0.436375575593 0.438255071927
261 0.434507767205 0.438626856874
262 0.434507767205 0.438350770399
263 0.434507767205 0.437695294723
264 0.434507767205 0.43782077464
265 0.434507767205 0.43868708358
Reset!!
266 0.434507767205 23.2134069608
267 0.434507767205 0.573272212521
268 0.434144110456 0.501750825361
269 0.434144110456 0.44750993533
270 0.434144110456 0.438058929223
271 0.434143989175 0.437183144658
272 0.434143989175 5.42964146576
273 0.433707254407 0.439187981062
274 0.433707254407 0.436715334811
275 0.433707254407 0.4371

489 0.41844332579 0.429603689695
490 0.417989980868 0.428294314971
491 0.417183677236 0.423994446593
492 0.417183677236 0.422479096103
493 0.417182254539 0.426675413934
494 0.416112875453 0.431470718584
495 0.415275829254 0.422228638725
496 0.415275829254 0.424876673607
497 0.415002191179 0.426822671971
498 0.415002191179 0.423985323412
499 0.415002191179 5.41430428321
500 0.415002191179 0.422223441
501 0.414685295568 0.43517204605
502 0.414521523088 0.425018495257
503 0.414521523088 0.422851473006
504 0.414521523088 0.421678935151
505 0.414521523088 0.432505304811
506 0.414455128576 0.444136382841
507 0.414455128576 0.441345341083
508 0.414455128576 0.466619727945
509 0.414356718758 0.430931897824
510 0.414356718758 0.422883845335
511 0.41434997181 0.429903949172
512 0.413674480662 0.424011191013
513 0.413674480662 0.421882758298
514 0.413674480662 0.428584527819
515 0.413674480662 0.418172096188
516 0.413674480662 5.41228985777
517 0.413674480662 0.42748694387
518 0.413674480662 0.42

732 0.400428162378 0.416355271721
733 0.400428162378 0.429366250676
734 0.400428162378 0.415065702634
735 0.400428162378 0.413541539149
736 0.400428162378 0.412823254167
737 0.400428162378 0.413841415155
738 0.400428162378 0.411055983042
739 0.400428162378 0.411681058426
740 0.400428162378 0.413044759054
741 0.400428162378 0.409423993954
742 0.400428162378 0.415191321136
743 0.400428162378 0.416067304096
744 0.400428162378 0.421025516873
745 0.400428162378 0.423992609289
746 0.400428162378 0.414898430932
747 0.400428162378 0.411349247106
748 0.400428162378 0.41747030625
749 0.400428162378 5.40089485114
750 0.400428162378 0.415372598783
751 0.400428162378 0.40682503836
752 0.400428162378 0.42695769967
753 0.400428162378 0.415424475416
754 0.400428162378 0.409410814373
755 0.400428162378 0.407279092259
756 0.400428162378 5.40710069958
757 0.40024934297 0.408165704453
758 0.400205084265 0.407780178792
759 0.400205084265 0.409918397314
760 0.400205084265 0.413252024514
761 0.400205084265 0

975 0.389868922886 0.402256447476
976 0.389834876917 0.394090056437
977 0.389834876917 0.393192667751
Reset!!
978 0.389834876917 38.3238361975
979 0.389834876917 0.541258845292
980 0.389606927091 0.463357102756
981 0.389606927091 0.423007242538
982 0.389606927091 0.399587775255
983 0.389606927091 0.394724158993
984 0.389606927091 0.3969647857
985 0.388370110833 0.393835047634
986 0.388370110833 0.394564196918
987 0.388370110833 0.405068279615
988 0.388370110833 0.39360877034
989 0.388286919081 0.411500759315
990 0.388286919081 0.39244206693
991 0.388286919081 0.399579787065
Reset!!
992 0.388286919081 33.6073208367
993 0.388286919081 0.591660391184
994 0.388286919081 0.520676399024
995 0.388286919081 0.408735603
996 0.388286919081 0.394165635922
997 0.388286919081 0.39385472117
998 0.388286919081 0.392821741872
999 0.388286919081 0.411469420439
1000 0.388286919081 0.428186341157
1001 0.388286919081 0.395429760513
1002 0.387155218849 0.391704531396
1003 0.387155218849 0.394782913589
1004

KeyboardInterrupt: 

In [7]:
str(gp.PrimitiveTree(pop[0]))


'minimum(add(add(protectedDiv(Sex, cos(cos(add(add(PClass, add(PClass, PClass)), mul(minimum(add(add(protectedDiv(PClass, cos(add(Fare, add(PClass, Cabin)))), sin(add(add(PClass, add(minimum(SibSp, Sex), add(PClass, PClass))), tanh(Parch)))), tanh(Age)), 3.141592653589793), minimum(SibSp, add(add(protectedDiv(PClass, protectedDiv(PClass, mul(PClass, 2.0))), add(Parch, Age)), minimum(protectedDiv(PClass, add(cos(sub(neg(tanh(Parch)), Sex)), neg(tanh(cos(sub(Age, Sex)))))), add(PClass, neg(protectedDiv(Age, PClass))))))))))), minimum(add(sin(sin(maximum(minimum(add(add(protectedDiv(cos(cos(add(Parch, PClass))), cos(cos(Fare))), sin(add(add(PClass, add(PClass, add(PClass, Cabin))), neg(Sex)))), add(PClass, add(minimum(3.141592653589793, minimum(add(PClass, Cabin), Fare)), mul(minimum(add(mul(PClass, PClass), tanh(Parch)), minimum(PClass, Fare)), 2.0)))), mul(PClass, mul(Embarked, PClass))), mul(add(PClass, Sex), minimum(add(protectedDiv(cos(cos(add(Parch, neg(protectedDiv(Age, 3.141592653

In [10]:
hof[0].fitness.values[0]

0.57932736864518519